## Automatic Speech Recognition Inference using Batch Endpoints

This sample shows how to deploy `automatic-speech-recognition` type models to a batch endpoint for inference.

### Task
`automatic-speech-recognition` (ASR) converts a speech signal to text, mapping a sequence of audio inputs to text outputs. Virtual assistants like Siri and Alexa use ASR models to help users everyday, and there are many other useful user-facing applications like live captioning and note-taking during meetings.

### Model
Models that can perform the `automatic-speech-recognition` task are tagged with `task: automatic-speech-recognition`. We will use the `openai-whisper-large` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name. If you don't find a model that suits your scenario or domain, you can discover and [import models from HuggingFace hub](../../import/import_model_into_registry.ipynb) and then use them for inference. 

### Inference data
We will use the [Librispeech ASR](https://huggingface.co/datasets/librispeech_asr/viewer/clean/test) dataset. \
You can use also use custom audio files stored on the cloud and verify inference.
- Most common audio formats (m4a, wav, flac, wma, mp3, etc.) are supported.

### Outline
* Set up pre-requisites.
* Pick a model to deploy.
* Prepare data for inference. 
* Deploy the model for batch inference.
* Run a batch inference job.
* Review inference predictions.
* Clean up resources.

### 1. Set up pre-requisites
* Install dependencies.
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry.
* Create or update compute.

In [ ]:
# Import packages used by the following code snippets
import csv
import json
import os
import requests
import time

import pandas as pd

from azure.ai.ml import Input, MLClient
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import (
    AmlCompute,
    BatchDeployment,
    BatchEndpoint,
    BatchRetrySettings,
    Model,
)

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group_name = "<RESOURCE_GROUP>"
workspace_name = "<WORKSPACE_NAME>"

#### Connect to workspace and registry using ML clients.

In [ ]:
try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group_name,
    workspace_name=workspace_name,
)
# The models, fine tuning pipelines, and environments are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(credential, registry_name="azureml")

#### Create a compute cluster.
Use the model card from the AzureML system registry to check the minimum required inferencing SKU, referenced as `size` below. If you already have a sufficient compute cluster, you can simply define the name in `compute_name` in the following code block.

In [ ]:
compute_name = "cpu-cluster"

In [ ]:
compute_cluster = AmlCompute(
    name=compute_name,
    description="An AML compute cluster",
    size="Standard_DS5_V2",
    min_instances=0,
    max_instances=3,
    idle_time_before_scale_down=120,
)  # 120 seconds

workspace_ml_client.begin_create_or_update(compute_cluster)

### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `automatic-speech-recognition` task. In this example, we use the `openai-whisper-large` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [ ]:
model_name = "openai-whisper-large"
model_version = "11"
foundation_model = registry_ml_client.models.get(model_name, model_version)
print(
    f"Using model name: {foundation_model.name}, version: {foundation_model.version}, id: {foundation_model.id} for inferencing."
)

### 3. Prepare data for inference

We will test with a small subset from the [Librispeech ASR](https://huggingface.co/datasets/librispeech_asr/viewer/clean/test) dataset, saving the sample in the `librispeech-dataset` folder.  The next few cells show basic data preparation:
* Download the data.
* Visualize some data rows.
* Save the data.

We want this sample to run quickly, so we are using a smaller dataset containing a fraction of the original.

In [ ]:
# Define directories and filenames as variables
dataset_dir = "librispeech-dataset"
test_datafile = "test_100.csv"

batch_dir = "batch"
batch_inputs_dir = os.path.join(batch_dir, "inputs")
batch_input_file = "batch_input.csv"
os.makedirs(dataset_dir, exist_ok=True)
os.makedirs(batch_dir, exist_ok=True)
os.makedirs(batch_inputs_dir, exist_ok=True)

#### 3.1 Download the data
We want to get the URLs for the audio files. To do this, pull down the data using `requests` instead of the Huggingface `datasets` module. The MLflow model's signature specifies the input should be a column named `"audio"` and a column named `"language"`.

In [ ]:
testdata = requests.get(
    "https://datasets-server.huggingface.co/first-rows?dataset=librispeech_asr&config=clean&split=test&offset=0&limit=100"
).text
testdata = json.loads(testdata)

In [ ]:
audio_urls_and_text = [
    (row["row"]["audio"][0]["src"], row["row"]["text"]) for row in testdata["rows"]
]

In [ ]:
test_df = pd.DataFrame(data=audio_urls_and_text, columns=["audio", "text"])

These are all spoken in English, so create a `language` column of all `'en'`.

In [ ]:
test_df["language"] = "en"
test_df.to_csv(os.path.join(".", dataset_dir, test_datafile), index=False)

#### 3.2 Visualize a few rows of data

In [ ]:
pd.set_option(
    "display.max_colwidth", 0
)  # Set the max column width to 0 to display the full text
test_df.head()

#### 3.3 Save the data
Save the input data to files of smaller batches for testing.

In [ ]:
batch_df = test_df[["audio", "language"]]

# Divide this into files of 10 rows each
batch_size_per_predict = 10
for i in range(0, len(batch_df), batch_size_per_predict):
    j = i + batch_size_per_predict
    batch_df[i:j].to_csv(
        os.path.join(batch_inputs_dir, str(i) + batch_input_file), quoting=csv.QUOTE_ALL
    )

# Check out the first and last file name created
input_files = os.listdir(batch_inputs_dir)
print(f"{input_files[0]} to {str(i)}{batch_input_file}.")

### 4. Deploy the model to a batch endpoint
Batch endpoints are endpoints that are used to do batch inferencing on large volumes of data over a period of time. The endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis. For more information on batch endpoints and deployments see [What are batch endpoints?](https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints?view=azureml-api-2#what-are-batch-endpoints).

* Create a batch endpoint.
* Create a batch deployment.
* Set the deployment as default; doing so allows invoking the endpoint without specifying the deployment's name.

#### Create the endpoint.

In [ ]:
# Endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
endpoint_name = "speech-recognition-" + str(timestamp)

endpoint = BatchEndpoint(
    name=endpoint_name,
    description="Batch endpoint for "
    + foundation_model.name
    + ", for automatic-speech-recognition task",
)
workspace_ml_client.begin_create_or_update(endpoint).result()

#### Create the deployment.

In [ ]:
deployment_name = "demo"

deployment = BatchDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=foundation_model.id,
    compute=compute_name,
    error_threshold=0,
    instance_count=1,
    logging_level="info",
    max_concurrency_per_instance=1,
    mini_batch_size=2,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=600),
)
workspace_ml_client.begin_create_or_update(deployment).result()

#### Set the deployment as default.

In [ ]:
endpoint = workspace_ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment_name
workspace_ml_client.begin_create_or_update(endpoint).wait()

endpoint = workspace_ml_client.batch_endpoints.get(endpoint_name)
print(f"The default deployment is {endpoint.defaults.deployment_name}")

### 5. Run a batch inference job
Invoke the batch endpoint with the input parameter pointing to the folder containing the batch inference input. This creates a pipeline job using the default deployment in the endpoint. Wait for the job to complete.

In [ ]:
input = Input(path=batch_inputs_dir, type=AssetTypes.URI_FOLDER)

job = workspace_ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, input=input
)

workspace_ml_client.jobs.stream(job.name)

### 6. Review inference predictions
Download the predictions from the job output and review the predictions using a dataframe.

In [ ]:
scoring_job = list(workspace_ml_client.jobs.list(parent_job_name=job.name))[0]

workspace_ml_client.jobs.download(
    name=scoring_job.name, download_path=batch_dir, output_name="score"
)

predictions_file = os.path.join(batch_dir, "named-outputs", "score", "predictions.csv")

# Load the batch predictions file with no headers into a dataframe and set your column names
score_df = pd.read_csv(
    predictions_file,
    header=None,
    names=["row_number_per_file", "prediction", "batch_input_file_name"],
)
score_df.head()

Record the input file name and set the original index value in the `'index'` column for each input file. Join the `test_df` containing ground truth into the input dataframe.

In [ ]:
input_df = []
for file in input_files:
    input = pd.read_csv(os.path.join(batch_inputs_dir, file), index_col=0)
    input.reset_index(inplace=True)
    input["batch_input_file_name"] = file
    input.reset_index(names=["row_number_per_file"], inplace=True)
    input_df.append(input)
input_df = pd.concat(input_df)
input_df.set_index("index", inplace=True)
input_df = input_df.join(test_df.drop(columns=["audio", "language"]))

input_df.head()

Join the predictions with input data to compare them to ground truth.

In [ ]:
df = pd.merge(
    input_df, score_df, how="inner", on=["row_number_per_file", "batch_input_file_name"]
)

# Show the first few rows of the results
df.head(20)

### 7. Clean up resources
Batch endpoints use compute resources only when jobs are submitted. You can keep the batch endpoint for your reference without worrying about compute bills, or choose to delete the endpoint. If you created your compute cluster to have zero minimum instances and scale down soon after being idle, you won't be charged for an unused compute.

In [ ]:
workspace_ml_client.batch_endpoints.begin_delete(name=endpoint_name).result()
workspace_ml_client.compute.begin_delete(name=compute_name).result()